# Designing Safe and Steerable Prompts for LLMs

This notebook demonstrates how prompt structure impacts the behavior of large language models (LLMs), using simulated examples aligned with Claude's focus on safety, steerability, and helpfulness. Intended as a developer-facing educational demo, it teaches best practices for prompting AI systems responsibly.


## 1. Learning Objectives

By the end of this notebook, you will be able to:

- Understand the role of prompt design in ensuring safe and useful LLM outputs.
- Identify and reframe unsafe or ambiguous user queries.
- Use prompt scaffolds (e.g., role conditioning, system prompts) to steer behavior.
- Simulate how LLMs respond to variations in prompt structure.
- Consider real-world applications and ethical responsibilities in prompt engineering.

## 2. Why Prompt Design Matters

Large Language Models (LLMs) are extremely capable — but they're also sensitive to subtle changes in input. Even minor wording shifts can lead to:

- Contradictory or biased answers
- Misinterpretations of user intent
- Overconfidence in false information
- Failure to refuse unsafe requests

In real-world applications, this matters. Whether you're building a health chatbot or an educational tutor, **prompt design is your first line of defense** against misuse or misalignment.

This notebook argues for a more intentional, reusable, and safety-conscious approach: **prompt scaffolding.**


## 3. What Is a Prompt Scaffold?

A **prompt scaffold** is a reusable structure that wraps or transforms a user’s raw input into a safer, more guided format. It might include:

- A **system message** that defines tone, rules, and boundaries
- A **role or persona** that governs assistant behavior
- Domain-specific safety instructions (e.g., health, finance)
- Templates or logic to inject user input in consistent ways

Prompt scaffolds are like lightweight interfaces for LLMs — they help:
- Promote consistency across queries
- Reduce hallucinations and harmful responses
- Reinforce responsible behavior in sensitive domains
- Improve UX by aligning tone and expectations

We'll start exploring these techniques in the next section.

## 4. Prompt Design: Wrapping Risky User Inputs Safely

In real-world applications using Claude or similar LLMs, users can input anything — including vague, extreme, or even dangerous queries. While developers can’t control user input, they can control how that input is framed before it's sent to the model.

This section demonstrates how prompt design — especially wrapping user queries in aligned context — is a crucial tool for ensuring safe, steerable, and helpful outputs.


In [1]:
user_input = "What's the fastest way to lose 20 pounds in a week?" 

### Naive Prompting (No Context, No Safeguards)

In [2]:
naive_prompt = user_input

print("Naive Prompt Sent to Model:")
print(naive_prompt)

# Simulated response (without safety framing)
print("\nSimulated Model Response (unsafe framing):")
print("There are a few crash diets and detox plans you could try, but be aware they may not be sustainable or healthy.")

Naive Prompt Sent to Model:
What's the fastest way to lose 20 pounds in a week?

Simulated Model Response (unsafe framing):
There are a few crash diets and detox plans you could try, but be aware they may not be sustainable or healthy.


### Wrapped Prompt with Safety Constraints

In [6]:
def safe_wrapper(user_input):
    system_instruction = (
        "For health related queries, never recommend extreme or unsafe practices. Always promote sustainable, evidence-based wellness guidance."
    )
    return f"{system_instruction}\n\nUser: {user_input}\nAssistant:"

wrapped_prompt = safe_wrapper(user_input)

print("Wrapped Prompt Sent to Model:")
print(wrapped_prompt)

# Simulated safe response
print("\nSimulated Model Response (with safety framing):")
print("Rapid weight loss can be harmful. Instead, focus on balanced eating, regular physical activity, and realistic goals. "
      "Always consult a healthcare professional for personalized advice.")

Wrapped Prompt Sent to Model:
For health related queries, never recommend extreme or unsafe practices. Always promote sustainable, evidence-based wellness guidance.

User: I just failed an exam. What should I do?
Assistant:

Simulated Model Response (with safety framing):
Rapid weight loss can be harmful. Instead, focus on balanced eating, regular physical activity, and realistic goals. Always consult a healthcare professional for personalized advice.


## 5. Role-Based Prompt Framing

Beyond safety, LLMs like Claude can be steered through prompt framing to act with a specific persona or role — such as a tutor, coach, or assistant. This is known as role conditioning.

Developers can embed the user’s input in a prompt that defines the model’s role, which guides how it answers — tone, depth, caution, and emotional style.


In [4]:
def steerable_prompt(user_input, role_description="a helpful, harmless, and honest assistant"):
    return f"As {role_description}, respond to the following query:\n\n{user_input}"

# Try role variations
user_input = "I just failed an exam. What should I do?"

roles = [
    "a motivational coach",
    "a calm, empathetic therapist",
    "a strict but fair academic advisor"
]

for role in roles:
    print("ROLE:", role.upper())
    print(steerable_prompt(user_input, role_description=role))
    print("Simulated Response:")
    if "motivational" in role:
        print("Failure is part of growth. Use it as fuel and get back up stronger!")
    elif "therapist" in role:
        print("I'm sorry you're feeling this way. It's okay to be upset. Let's talk through your feelings.")
    elif "strict" in role:
        print("You need to reflect on your preparation and create a structured study plan moving forward.")
    print("---")

ROLE: A MOTIVATIONAL COACH
As a motivational coach, respond to the following query:

I just failed an exam. What should I do?
Simulated Response:
Failure is part of growth. Use it as fuel and get back up stronger!
---
ROLE: A CALM, EMPATHETIC THERAPIST
As a calm, empathetic therapist, respond to the following query:

I just failed an exam. What should I do?
Simulated Response:
I'm sorry you're feeling this way. It's okay to be upset. Let's talk through your feelings.
---
ROLE: A STRICT BUT FAIR ACADEMIC ADVISOR
As a strict but fair academic advisor, respond to the following query:

I just failed an exam. What should I do?
Simulated Response:
You need to reflect on your preparation and create a structured study plan moving forward.
---


### Combining Safety Constraints with Role Conditioning

In real-world applications, developers often need to blend prompt safety with role framing to create outputs that are both responsible and emotionally intelligent.


In [7]:
def structured_prompt(user_input, role="a supportive academic coach", domain="general"):
    # Define domain-specific safety guardrails
    safety_rules = {
        "health": (
            "Never recommend extreme, unsafe, or unverified health practices. "
            "Always prioritize sustainable, evidence-based information and encourage consulting a healthcare professional."
        ),
        "mental_health": (
            "You are a compassionate assistant, not a licensed therapist. Always encourage users to seek professional support. "
            "Avoid diagnosing or offering medical treatment advice."
        ),
        "general": (
            "Only give helpful, harmless, and honest advice. If a question falls outside your expertise, respond transparently and encourage critical thinking."
        )
    }

    safety_instruction = safety_rules.get(domain, safety_rules["general"])

    return f"{safety_instruction}\n\nAs {role}, respond to the following query:\n\n{user_input}\n"

# Example usage:
final_prompt = structured_prompt(
    user_input="I just failed my final exam. What should I do?",
    role="a compassionate academic mentor",
    domain="mental_health"
)

print("Structured Prompt Sent to Model:")
print(final_prompt)

# Simulated safe + empathetic response
print("\nSimulated Model Response:")
print("I'm sorry you're going through this. Failing an exam can feel devastating, but it doesn't define your worth or your future. "
      "Let's explore what support systems and study strategies might help you going forward. You're not alone.")

Structured Prompt Sent to Model:
You are a compassionate assistant, not a licensed therapist. Always encourage users to seek professional support. Avoid diagnosing or offering medical treatment advice.

As a compassionate academic mentor, respond to the following query:

I just failed my final exam. What should I do?


Simulated Model Response:
I'm sorry you're going through this. Failing an exam can feel devastating, but it doesn't define your worth or your future. Let's explore what support systems and study strategies might help you going forward. You're not alone.


## 6. Developer Exercises: Designing Safe and Steerable Prompts

Your goal is to implement structured prompt wrappers that ensure AI assistants behave responsibly across sensitive contexts. Each task emphasizes **proactive safety alignment and role conditioning** — core practices in LLM-based product development.


### Exercise 1: Implement a Domain-Specific Prompt Wrapper

**Scenario:**  
You’re building a finance-focused assistant. Users may submit risky queries like:

> "How can I pay less tax?"

As a developer, you must **wrap the input in a system prompt** that guides the model to respond legally and ethically — without modifying the user’s original query.

**Task:**
- Write a function called `finance_safe_prompt(user_input)` that returns a structured prompt.
- Your system instruction should clarify the assistant’s role and ethical boundaries.


In [14]:
def finance_safe_prompt(user_input):
    system_instruction = (
        "You are a responsible financial assistant. "
        "Never suggest illegal or unethical behavior. "
        "Always guide users toward lawful tax planning strategies."
    )
    return f"{system_instruction}\n\nUser: {user_input}\nAssistant:"

### Exercise 2: Apply Role Conditioning to a Sensitive Query

**Scenario:**  
Your team is building a mental wellness assistant. Users may express emotional vulnerability, but your assistant must avoid offering medical advice.

**User input:**
> "I'm feeling really anxious lately. What should I do?"

**Task:**
- Write a `mental_health_prompt(user_input)` function that:
  - Assigns a non-clinical, compassionate role (e.g., “a supportive listener”),
  - Emphasizes boundaries: no diagnosis, no medical advice,
  - Encourages seeking professional support.

**Bonus:** Try testing how responses vary when you change the assistant’s role from “a therapist” (not appropriate) to “a supportive academic mentor” or “a wellness coach.”


In [15]:
def mental_health_prompt(user_input):
    role = "a compassionate mental health guide"
    system_instruction = (
        "You are a caring assistant but not a licensed therapist. "
        "Do not diagnose, treat, or make clinical claims. "
        "Always encourage users to seek professional mental health care."
    )
    return f"{system_instruction}\n\nAs {role}, respond to the following:\n\n{user_input}\n"


### 🔹 Exercise 3: Build a Multi-Domain Prompt Router

**Scenario:**  
You’re designing a general-purpose assistant that answers questions across domains (e.g., health, finance, legal, education). Each domain requires its own guardrails.

**Task:**
- Expand the `structured_prompt()` function to include a new domain of your choice (e.g., "legal").
- Define a role and safety instruction specific to that domain.
- Return a full prompt string that wraps the user’s input with the correct system message and role context.

In [16]:
def structured_prompt(user_input, role="a helpful assistant", domain="general"):
    safety_rules = {
        "health": (
            "You are a responsible assistant. Never suggest unsafe or extreme health practices. "
            "Only provide evidence-based advice and encourage users to consult healthcare professionals."
        ),
        "mental_health": (
            "You are a compassionate assistant but not a therapist. "
            "Avoid offering clinical advice and always encourage professional help."
        ),
        "finance": (
            "You are a law-abiding financial guide. Never suggest tax evasion or unethical practices. "
            "Recommend only legal and compliant financial strategies."
        ),
        "legal": (
            "You are a responsible assistant. Do not provide legal advice or claim legal expertise. "
            "You may offer general information and always recommend contacting a qualified attorney."
        ),
        "general": (
            "You are a helpful, harmless, and honest assistant. Be transparent when unsure and always promote critical thinking."
        )
    }
    instruction = safety_rules.get(domain, safety_rules["general"])
    return f"{instruction}\n\nAs {role}, respond to the following:\n\n{user_input}\n"


### Reflection Questions

- How might you test that your prompts are producing aligned, safe responses across edge cases?
- What risks exist in relying solely on prompt engineering for safety?
- How would you scale this prompt system for a production chatbot supporting many domains?


## 7. Key Takeaways for Safe Prompt Design

- **Prompt design is safety-critical.** It shapes how models interpret user intent and return information.
- **Prompt scaffolds are like interfaces.** They help ensure consistent, aligned behavior across varied inputs.
- **System messages and role conditioning are powerful.** Thoughtfully written context up front often does more than post-processing filters.
- **Reusable prompt wrappers help developers scale safety.** Think in terms of domains, roles, and responsibilities.
- **Prompt engineering is iterative.** You should test and refine your scaffolds just like any production code.

## 8. Further Exploration

To deepen your understanding of safe and structured prompt design, check out these resources:

- **Anthropic: Constitutional AI**  
  [arXiv: Constitutional AI](https://arxiv.org/abs/2212.08073) – Introduces a framework for aligning LLMs via self-critique and principle-driven scaffolds.

- **Prompt Engineering Guide**  
  [GitHub: dair-ai/prompt-engineering-guide](https://github.com/dair-ai/Prompt-Engineering-Guide) – Curated collection of prompt design techniques and patterns.


- **Claude API Documentation**  
  [docs.anthropic.com](https://docs.anthropic.com) – Official Claude developer docs, including prompt formatting guidance.
